In [1]:
import os
os.chdir('/home/reffert/DeepAR_InfluenzaForecast')
from PythonFiles.Configuration import Configuration
from PythonFiles.HpTuning import get_data, objectiveFNN
from ray.air import session
from ray import tune
data_splits_dict = get_data(truncate=False, with_features=True)[0]

In [ ]:
train = data_splits_dict["with_features_2001"][0]
test = data_splits_dict["with_features_2001"][1]
configuration = Configuration()
hp_search_space = {
    "num_hidden_dimensions": tune.grid_search([[40,40],[80,80],[20,20]]),
    "context_length":tune.grid_search([4, 52, 104]),
    "epochs":tune.grid_search([100, 200]),
    "prediction_length" : tune.grid_search([4]), 
    "num_batches_per_epoch":tune.grid_search([50]),
    "batch_normalization":tune.grid_search([True, False]),
    "batch_size":tune.grid_search([32]),
    }

tuner = tune.Tuner(
    tune.with_parameters(objectiveFNN, train=train, test=test, configuration=configuration),
    tune_config=tune.TuneConfig(
        num_samples = 5,
        metric="mean_WIS",
        mode="min",
        max_concurrent_trials=22,
    ),
    param_space=hp_search_space,
)
results = tuner.fit()

print("Best hyperparameters found were: ", results.get_best_result().config)

results_df = results.get_dataframe()
print(results_df)
results_df.to_csv("FNN_Hyperparameter_results_29_05.csv")

100%|██████████| 50/50 [00:17<00:00,  2.87it/s, epoch=45/100, avg_epoch_loss=0.773]
(objectiveFNN pid=1282511) 
  0%|          | 0/50 [00:00<?, ?it/s]m 
Running evaluation: 0it [00:00, ?it/s]m 
  0%|          | 0/50 [00:00<?, ?it/s]m 
Running evaluation: 41922it [00:02, 16619.93it/s]
  0%|          | 0/50 [00:00<?, ?it/s]m 
Running evaluation: 0it [00:00, ?it/s]m 
  0%|          | 0/50 [00:00<?, ?it/s]m 
Running evaluation: 41922it [00:04, 9728.17it/s]
  0%|          | 0/50 [00:00<?, ?it/s]m 
(objectiveFNN pid=1282720) /home/reffert/.conda/envs/TEST_ENV/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1181: UserWarning: Warning: converting a masked element to nan.
(objectiveFNN pid=1282720)   return arr.astype(dtype, copy=True)
  0%|          | 0/50 [00:00<?, ?it/s]m 


Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,mean_WIS,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
objectiveFNN_1d7f9_00000,2023-05-29_14-35-42,True,,eaf33ae52da543b2a66614e94ea35848,"0_batch_normalization=True,batch_size=32,context_length=4,epochs=100,num_batches_per_epoch=50,num_hidden_dimensions=40_40,prediction_length=4",econ-stat-rr01,1,583.151,172.22.1.197,1282303,1362.21,1362.21,1362.21,1685363742,0,,1,1d7f9_00000,0.0414429
objectiveFNN_1d7f9_00001,2023-05-29_14-38-34,True,,51053c5fd3634cc2a3b67462027bfe58,"1_batch_normalization=False,batch_size=32,context_length=4,epochs=100,num_batches_per_epoch=50,num_hidden_dimensions=40_40,prediction_length=4",econ-stat-rr01,1,511.48,172.22.1.197,1282397,1516.57,1516.57,1516.57,1685363914,0,,1,1d7f9_00001,0.0637748
objectiveFNN_1d7f9_00002,2023-05-29_14-37-06,True,,fdaf670097db45b5bc1a2b63c7114015,"2_batch_normalization=True,batch_size=32,context_length=52,epochs=100,num_batches_per_epoch=50,num_hidden_dimensions=40_40,prediction_length=4",econ-stat-rr01,1,456.48,172.22.1.197,1282511,1408.01,1408.01,1408.01,1685363826,0,,1,1d7f9_00002,0.0461538
objectiveFNN_1d7f9_00003,2023-05-29_14-37-29,True,,34263220be274721a77b36a37ec09b26,"3_batch_normalization=False,batch_size=32,context_length=52,epochs=100,num_batches_per_epoch=50,num_hidden_dimensions=40_40,prediction_length=4",econ-stat-rr01,1,405.522,172.22.1.197,1282628,1412.24,1412.24,1412.24,1685363849,0,,1,1d7f9_00003,0.0425601
objectiveFNN_1d7f9_00004,2023-05-29_14-35-35,True,,23b7186859014e8bb619702ae5fd313c,"4_batch_normalization=True,batch_size=32,context_length=104,epochs=100,num_batches_per_epoch=50,num_hidden_dimensions=40_40,prediction_length=4",econ-stat-rr01,1,429.182,172.22.1.197,1282720,1276.24,1276.24,1276.24,1685363735,0,,1,1d7f9_00004,0.0445077
objectiveFNN_1d7f9_00005,2023-05-29_14-37-08,True,,295aaed8214e4bed8a72ecec61470a78,"5_batch_normalization=False,batch_size=32,context_length=104,epochs=100,num_batches_per_epoch=50,num_hidden_dimensions=40_40,prediction_length=4",econ-stat-rr01,1,417.343,172.22.1.197,1282815,1348.14,1348.14,1348.14,1685363828,0,,1,1d7f9_00005,0.0399866
objectiveFNN_1d7f9_00012,2023-05-29_14-41-52,True,,aaa6b91abfff4edca56475d95546f0bc,"12_batch_normalization=True,batch_size=32,context_length=4,epochs=100,num_batches_per_epoch=50,num_hidden_dimensions=80_80,prediction_length=4",econ-stat-rr01,1,541.851,172.22.1.197,1283624,1481.85,1481.85,1481.85,1685364112,0,,1,1d7f9_00012,0.0833488
objectiveFNN_1d7f9_00013,2023-05-29_14-44-04,True,,7a0064c3f6ce4ef288aee82515e1e8f4,"13_batch_normalization=False,batch_size=32,context_length=4,epochs=100,num_batches_per_epoch=50,num_hidden_dimensions=80_80,prediction_length=4",econ-stat-rr01,1,525.701,172.22.1.197,1283757,1585.39,1585.39,1585.39,1685364244,0,,1,1d7f9_00013,0.0536356
objectiveFNN_1d7f9_00014,2023-05-29_14-45-34,True,,1b5e786f972c47cd9a1df4bc158b4f4c,"14_batch_normalization=True,batch_size=32,context_length=52,epochs=100,num_batches_per_epoch=50,num_hidden_dimensions=80_80,prediction_length=4",econ-stat-rr01,1,419.832,172.22.1.197,1283902,1649.02,1649.02,1649.02,1685364334,0,,1,1d7f9_00014,0.0441487
objectiveFNN_1d7f9_00015,2023-05-29_14-39-23,True,,0d2c317313864392b8969ffa3a774497,"15_batch_normalization=False,batch_size=32,context_length=52,epochs=100,num_batches_per_epoch=50,num_hidden_dimensions=80_80,prediction_length=4",econ-stat-rr01,1,414.353,172.22.1.197,1284049,1255.7,1255.7,1255.7,1685363963,0,,1,1d7f9_00015,0.0544794


  0%|          | 0/50 [00:00<?, ?it/s]m 
Running evaluation: 0it [00:00, ?it/s]m 
  0%|          | 0/50 [00:00<?, ?it/s]m 
Running evaluation: 41922it [00:04, 9359.19it/s]
  0%|          | 0/50 [00:00<?, ?it/s]m 
(objectiveFNN pid=1282303) /home/reffert/.conda/envs/TEST_ENV/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1181: UserWarning: Warning: converting a masked element to nan.
(objectiveFNN pid=1282303)   return arr.astype(dtype, copy=True)
  0%|          | 0/50 [00:00<?, ?it/s]m 
Running evaluation: 0it [00:00, ?it/s]m 
  0%|          | 0/50 [00:00<?, ?it/s]m 
Running evaluation: 41922it [00:04, 9333.66it/s]
  0%|          | 0/50 [00:00<?, ?it/s]m 
Running evaluation: 0it [00:00, ?it/s]m 
  0%|          | 0/50 [00:00<?, ?it/s]m 
Running evaluation: 41922it [00:08, 4908.27it/s]
 72%|███████▏  | 36/50 [00:33<00:13,  1.04it/s, epoch=97/200, avg_epoch_loss=0.763]
Running evaluation: 0it [00:00, ?it/s]m 
  0%|          | 0/50 [00:00<?, ?it/s]m 
(objectiveFNN pid=1282815) /ho